# Astro II — Comoving Coordinates (Proper vs Comoving)

**Goal:** build intuition for what “expansion of space” means in an FLRW-like universe.

Key relation:
$
\mathbf{x}_{\rm proper}(t) = a(t)\,\boldsymbol{\chi}(t),
$
where:
- $(a(t))$ is the **scale factor**
- $(\boldsymbol{\chi})$ are **comoving coordinates**
- $(\mathbf{x}_{\rm proper})$ are **proper (physical) coordinates**

If galaxies are “comoving” with the expansion, then $(\boldsymbol{\chi})$ is constant and all growth is in $(a(t)$).

---

## What to do
1. Use the **time slider** to change \(t\) and watch galaxies separate in **proper** coordinates.
2. Switch to **comoving** view: galaxies mostly sit still (coordinates absorb the expansion).
3. Give one galaxy a **peculiar velocity** and observe:
   - in proper space, it drifts relative to the Hubble flow
   - in comoving space, its comoving coordinate changes (because $(d\chi/dt = v_{\rm pec}/a)$).

This separates **Hubble flow** from **peculiar motion**.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import ipywidgets as widgets
from IPython.display import display, clear_output

# Use a clean, modern style
plt.style.use('seaborn-v0_8-whitegrid')

# Reproducibility
rng = np.random.default_rng(7)

# "Galaxies" in comoving coordinates in a square box
N = 250
chi = rng.uniform(-1.0, 1.0, size=(N, 2))

# One special galaxy for peculiar motion
special_idx = 0
chi0_special = chi[special_idx].copy()

# Store trajectory for trail visualization
trajectory_length = 30
trajectory_proper = []
trajectory_comoving = []

def a_of_t(t, model="matter+Lambda"):
    """Toy scale-factor models (pedagogical, not a full cosmology solver)."""
    if model == "linear":
        return np.clip(1.0 + 0.5*t, 0.05, 10.0)
    if model == "matter":
        t0 = 1.0
        return np.clip(((t + t0)/t0)**(2/3), 0.05, 10.0)
    if model == "deSitter":
        H = 0.6
        return np.clip(np.exp(H*t), 0.05, 10.0)

    # default: smooth transition from matter-like to accelerated expansion
    t0 = 1.0
    a_m = ((t + t0)/t0)**(2/3)
    a_d = np.exp(0.45*t)
    w = 1/(1 + np.exp(-2*(t-1.0)))
    a = (1-w)*a_m + w*a_d
    return np.clip(a, 0.05, 10.0)

def integrate_comoving_shift(t_grid, v_pec, model):
    """For the special galaxy: dχ/dt = v_pec / a(t)."""
    t_grid = np.asarray(t_grid)
    a_grid = a_of_t(t_grid, model=model)
    integrand = v_pec / a_grid

    i0 = np.argmin(np.abs(t_grid - 0.0))
    cum = np.zeros_like(t_grid)

    for i in range(i0+1, len(t_grid)):
        dt = t_grid[i] - t_grid[i-1]
        cum[i] = cum[i-1] + 0.5*dt*(integrand[i] + integrand[i-1])

    for i in range(i0-1, -1, -1):
        dt = t_grid[i+1] - t_grid[i]
        cum[i] = cum[i+1] - 0.5*dt*(integrand[i] + integrand[i+1])

    return cum

def plot_universe_dual(t=0.0, model="matter+Lambda", v_pec=0.0, show_grid=True, show_trail=True):
    """Side-by-side view: proper and comoving coordinates with fixed axes."""
    global trajectory_proper, trajectory_comoving
    
    a = a_of_t(t, model=model)

    t_grid = np.linspace(-1.0, 3.0, 801)
    cum = integrate_comoving_shift(t_grid, v_pec=v_pec, model=model)
    shift = np.interp(t, t_grid, cum)

    chi_now = chi.copy()
    chi_now[special_idx, 0] = chi0_special[0] + shift
    chi_now[special_idx, 1] = chi0_special[1]

    # Proper coordinates
    x_proper = a * chi_now
    
    # Comoving coordinates
    x_comoving = chi_now
    
    # Update trajectories
    trajectory_proper.append(x_proper[special_idx].copy())
    trajectory_comoving.append(x_comoving[special_idx].copy())
    
    if len(trajectory_proper) > trajectory_length:
        trajectory_proper.pop(0)
        trajectory_comoving.pop(0)

    # Create figure with 3 subplots: proper, comoving, and scale factor
    fig = plt.figure(figsize=(16, 6))
    gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.4], height_ratios=[1, 0.15], hspace=0.3, wspace=0.3)
    
    ax_proper = fig.add_subplot(gs[0, 0])
    ax_comoving = fig.add_subplot(gs[0, 1])
    ax_scale = fig.add_subplot(gs[0, 2])
    ax_info = fig.add_subplot(gs[1, :])
    ax_info.axis('off')
    
    # Color scheme
    bg_color = '#A8B8C8'  # Soft blue-gray for background galaxies
    special_color = '#FF6B35'  # Bright orange for special galaxy
    trail_color = '#FFB84D'  # Lighter orange for trail
    
    # ===== LEFT PANEL: PROPER COORDINATES =====
    ax_proper.scatter(x_proper[:, 0], x_proper[:, 1], s=20, alpha=0.5, 
                      c=bg_color, edgecolors='none')
    
    # Trail for special galaxy
    if show_trail and len(trajectory_proper) > 1:
        trail = np.array(trajectory_proper)
        for i in range(len(trail) - 1):
            alpha_val = (i + 1) / len(trail) * 0.4
            ax_proper.plot(trail[i:i+2, 0], trail[i:i+2, 1], 
                          color=trail_color, alpha=alpha_val, linewidth=2)
    
    ax_proper.scatter(x_proper[special_idx, 0], x_proper[special_idx, 1], 
                     s=300, marker='*', c=special_color, edgecolors='white', 
                     linewidths=1.5, zorder=5)
    
    ax_proper.set_title(r"$\mathbf{Proper}$ $\mathbf{Coordinates:}$ $\mathbf{x} = a(t)\boldsymbol{\chi}$", 
                       fontsize=14, fontweight='bold', pad=15)
    ax_proper.set_xlabel(r"$x_{\rm proper}$", fontsize=13)
    ax_proper.set_ylabel(r"$y_{\rm proper}$", fontsize=13)
    ax_proper.set_aspect("equal", adjustable="box")
    
    # FIXED axes for proper view - scale with maximum expected a(t)
    max_a = a_of_t(3.0, model=model)
    L_proper = 1.2 * max_a
    ax_proper.set_xlim(-L_proper, L_proper)
    ax_proper.set_ylim(-L_proper, L_proper)
    
    if show_grid:
        ax_proper.grid(True, alpha=0.12, linestyle='--', linewidth=0.5)
    else:
        ax_proper.grid(False)
    
    # Annotation
    if abs(v_pec) < 0.01:
        motion_text = "Pure Hubble Flow"
    else:
        motion_text = f"+ Peculiar Motion"
    ax_proper.text(0.03, 0.97, motion_text, transform=ax_proper.transAxes, 
                  fontsize=11, va='top', bbox=dict(boxstyle='round', 
                  facecolor='white', alpha=0.8))
    
    # ===== MIDDLE PANEL: COMOVING COORDINATES =====
    ax_comoving.scatter(x_comoving[:, 0], x_comoving[:, 1], s=20, alpha=0.5, 
                       c=bg_color, edgecolors='none')
    
    # Trail for special galaxy
    if show_trail and len(trajectory_comoving) > 1:
        trail = np.array(trajectory_comoving)
        for i in range(len(trail) - 1):
            alpha_val = (i + 1) / len(trail) * 0.4
            ax_comoving.plot(trail[i:i+2, 0], trail[i:i+2, 1], 
                           color=trail_color, alpha=alpha_val, linewidth=2)
    
    ax_comoving.scatter(x_comoving[special_idx, 0], x_comoving[special_idx, 1], 
                       s=300, marker='*', c=special_color, edgecolors='white', 
                       linewidths=1.5, zorder=5)
    
    ax_comoving.set_title(r"$\mathbf{Comoving}$ $\mathbf{Coordinates:}$ $\boldsymbol{\chi}$ $\mathbf{(Expansion}$ $\mathbf{Incorporated)}$", 
                         fontsize=14, fontweight='bold', pad=15)
    ax_comoving.set_xlabel(r"$\chi_x$", fontsize=13)
    ax_comoving.set_ylabel(r"$\chi_y$", fontsize=13)
    ax_comoving.set_aspect("equal", adjustable="box")
    
    # FIXED axes for comoving view
    L_comoving = 1.2
    ax_comoving.set_xlim(-L_comoving, L_comoving)
    ax_comoving.set_ylim(-L_comoving, L_comoving)
    
    if show_grid:
        ax_comoving.grid(True, alpha=0.12, linestyle='--', linewidth=0.5)
    else:
        ax_comoving.grid(False)
    
    # Annotation
    if abs(v_pec) < 0.01:
        motion_text = "Galaxies Fixed"
    else:
        motion_text = r"$d\chi/dt = v_{\rm pec}/a$"
    ax_comoving.text(0.03, 0.97, motion_text, transform=ax_comoving.transAxes, 
                    fontsize=11, va='top', bbox=dict(boxstyle='round', 
                    facecolor='white', alpha=0.8))
    
    # ===== RIGHT PANEL: SCALE FACTOR a(t) =====
    t_plot = np.linspace(-1.0, 3.0, 300)
    a_plot = a_of_t(t_plot, model=model)
    
    ax_scale.plot(t_plot, a_plot, linewidth=2.5, color='#4A90E2', label=r'$a(t)$')
    ax_scale.scatter([t], [a], s=150, c='#FF6B35', edgecolors='white', 
                    linewidths=2, zorder=5, marker='o')
    
    ax_scale.set_xlabel(r"$t$", fontsize=12)
    ax_scale.set_ylabel(r"$a(t)$", fontsize=12)
    ax_scale.set_title("Scale Factor", fontsize=12, fontweight='bold', pad=10)
    ax_scale.grid(True, alpha=0.2, linestyle='--', linewidth=0.5)
    ax_scale.set_xlim(-1.0, 3.0)
    ax_scale.set_ylim(0, max(a_plot) * 1.1)
    
    # ===== INFO PANEL =====
    info_text = (
        f"Time: $t = {t:.2f}$  |  "
        f"Scale Factor: $a(t) = {a:.2f}$  |  "
        f"Model: {model}  |  "
        f"Peculiar Velocity: $v_{{\\rm pec}} = {v_pec:.2f}$"
    )
    ax_info.text(0.5, 0.5, info_text, ha='center', va='center', 
                fontsize=12, bbox=dict(boxstyle='round', facecolor='#E8F4F8', alpha=0.8))
    
    # plt.tight_layout()
    plt.show()

# ===== WIDGETS =====
play_button = widgets.Play(
    value=0,
    min=-100,
    max=400,
    step=1,
    interval=50,  # 50ms between frames = 20 fps (smooth animation)
    description="Play",
    disabled=False
)

t_slider = widgets.FloatSlider(
    value=0.0, 
    min=-1.0, 
    max=3.0, 
    step=0.01, 
    description="Time (t)", 
    readout_format=".2f",
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

# Link play button to slider
widgets.jslink((play_button, 'value'), (t_slider, 'value'))

model_dropdown = widgets.Dropdown(
    options=[("Toy ΛCDM-ish", "matter+Lambda"),
             ("Matter-dominated", "matter"),
             ("de Sitter (exponential)", "deSitter"),
             ("Linear expansion", "linear")],
    value="matter+Lambda",
    description="Model:",
    style={'description_width': '80px'}
)

vpec_slider = widgets.FloatSlider(
    value=0.0, 
    min=-1.5, 
    max=1.5, 
    step=0.01, 
    description="v_pec:", 
    readout_format=".2f",
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

grid_check = widgets.Checkbox(value=True, description="Show Grid")
trail_check = widgets.Checkbox(value=True, description="Show Trail")

# Preset buttons
button_pure_hubble = widgets.Button(description="Pure Hubble Flow", button_style='info')
button_pos_pec = widgets.Button(description="+ Peculiar Velocity", button_style='success')
button_neg_pec = widgets.Button(description="− Peculiar Velocity", button_style='warning')
button_reset = widgets.Button(description="Reset", button_style='danger')

def on_pure_hubble_click(b):
    vpec_slider.value = 0.0
    t_slider.value = 0.0
    trajectory_proper.clear()
    trajectory_comoving.clear()

def on_pos_pec_click(b):
    vpec_slider.value = 0.8
    t_slider.value = 0.0
    trajectory_proper.clear()
    trajectory_comoving.clear()

def on_neg_pec_click(b):
    vpec_slider.value = -0.8
    t_slider.value = 0.0
    trajectory_proper.clear()
    trajectory_comoving.clear()

def on_reset_click(b):
    vpec_slider.value = 0.0
    t_slider.value = 0.0
    model_dropdown.value = "matter+Lambda"
    trajectory_proper.clear()
    trajectory_comoving.clear()

button_pure_hubble.on_click(on_pure_hubble_click)
button_pos_pec.on_click(on_pos_pec_click)
button_neg_pec.on_click(on_neg_pec_click)
button_reset.on_click(on_reset_click)

# Layout
ui = widgets.VBox([
    widgets.HTML("<h3>Interactive Universe Expansion Visualizer</h3>"),
    widgets.HBox([play_button, t_slider]),
    widgets.HBox([model_dropdown, vpec_slider]),
    widgets.HBox([grid_check, trail_check]),
    widgets.HTML("<b>Quick Presets:</b>"),
    widgets.HBox([button_pure_hubble, button_pos_pec, button_neg_pec, button_reset]),
])

out = widgets.Output()

def _update(*args):
    with out:
        clear_output(wait=True)
        plot_universe_dual(
            t=t_slider.value,
            model=model_dropdown.value,
            v_pec=vpec_slider.value,
            show_grid=grid_check.value,
            show_trail=trail_check.value
        )

# Observe changes
for w in [t_slider, model_dropdown, vpec_slider, grid_check, trail_check]:
    w.observe(_update, names="value")

display(ui, out)
_update()


Output()

### Think about this:
1. In **comoving coordinates**, why do most galaxies “stand still” even though the universe expands?
2. If a galaxy has constant **peculiar velocity** $(v_{\rm pec})$, why does its comoving coordinate change as $(d\chi/dt=v_{\rm pec}/a(t))$?
3. In the proper view, why does recession speed roughly scale with distance (Hubble flow)?


---

## 3D Expanding Sphere Visualization

Now let's see how comoving vs proper coordinates work on a **2D surface of an expanding sphere** — a classic analogy for our expanding 3D universe!

**What you'll see:**
- **Left:** Proper coordinates — the sphere radius grows with $a(t)$, grid lines stretch apart
- **Right:** Comoving coordinates — sphere stays fixed size, grid is constant
- **Galaxy marker (red):** Tracks a point on the sphere with optional peculiar motion

This demonstrates that in comoving space, "expansion" is factored out — only peculiar motion changes positions.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_sphere_grid(n_theta=20, n_phi=20):
    """Create latitude/longitude grid on unit sphere."""
    theta = np.linspace(0, np.pi, n_theta)
    phi = np.linspace(0, 2*np.pi, n_phi)
    
    # Grid lines: constant latitude
    lat_lines = []
    for t in theta[1:-1]:  # Skip poles
        x = np.sin(t) * np.cos(phi)
        y = np.sin(t) * np.sin(phi)
        z = np.cos(t) * np.ones_like(phi)
        lat_lines.append((x, y, z))
    
    # Grid lines: constant longitude
    lon_lines = []
    for p in phi[::2]:  # Every other longitude line
        x = np.sin(theta) * np.cos(p)
        y = np.sin(theta) * np.sin(p)
        z = np.cos(theta)
        lon_lines.append((x, y, z))
    
    return lat_lines, lon_lines

def plot_expanding_sphere_3d(t=0.0, model="matter+Lambda", theta_pec=0.0, phi_pec=0.0):
    """
    Side-by-side 3D visualization of expanding sphere.
    
    Left: Proper coordinates (sphere grows with a(t))
    Right: Comoving coordinates (sphere fixed at radius 1)
    
    theta_pec, phi_pec: angular peculiar velocities (radians per time unit)
    """
    a = a_of_t(t, model=model)
    
    # Initial galaxy position (comoving coordinates: theta, phi on unit sphere)
    theta0 = np.pi/3  # 60 degrees from north pole
    phi0 = np.pi/4    # 45 degrees longitude
    
    # Integrate peculiar motion
    theta_galaxy = theta0 + theta_pec * t
    phi_galaxy = phi0 + phi_pec * t
    
    # Clamp theta to valid range
    theta_galaxy = np.clip(theta_galaxy, 0.1, np.pi - 0.1)
    
    # Comoving position on unit sphere
    x_com = np.sin(theta_galaxy) * np.cos(phi_galaxy)
    y_com = np.sin(theta_galaxy) * np.sin(phi_galaxy)
    z_com = np.cos(theta_galaxy)
    
    # Proper position (scaled by a(t))
    x_prop = a * x_com
    y_prop = a * y_com
    z_prop = a * z_com
    
    # Create grid
    lat_lines, lon_lines = create_sphere_grid(n_theta=15, n_phi=24)
    
    # Create figure with two 3D subplots
    fig = plt.figure(figsize=(16, 7))
    
    # ===== LEFT: PROPER COORDINATES =====
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Draw sphere surface (wireframe)
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x_sphere = a * np.outer(np.cos(u), np.sin(v))
    y_sphere = a * np.outer(np.sin(u), np.sin(v))
    z_sphere = a * np.outer(np.ones(np.size(u)), np.cos(v))
    
    ax1.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.15, color='lightblue', 
                     linewidth=0, antialiased=True, shade=False)
    
    # Draw coordinate grid
    for x, y, z in lat_lines:
        ax1.plot(a*x, a*y, a*z, color='#4A90E2', alpha=0.4, linewidth=1.2)
    for x, y, z in lon_lines:
        ax1.plot(a*x, a*y, a*z, color='#4A90E2', alpha=0.4, linewidth=1.2)
    
    # Draw galaxy
    ax1.scatter([x_prop], [y_prop], [z_prop], color='#FF6B35', s=300, 
               marker='*', edgecolors='white', linewidths=2, zorder=10)
    
    # Draw line from center to galaxy
    ax1.plot([0, x_prop], [0, y_prop], [0, z_prop], 
            color='#FF6B35', linewidth=2, linestyle='--', alpha=0.6)
    
    # Labels and formatting
    ax1.set_title(r"$\mathbf{Proper}$ $\mathbf{Coordinates}$" + f"\n" + 
                 r"Radius $= a(t) = $" + f"{a:.2f}", 
                 fontsize=14, fontweight='bold', pad=15)
    ax1.set_xlabel(r'$x_{\rm proper}$', fontsize=11, labelpad=8)
    ax1.set_ylabel(r'$y_{\rm proper}$', fontsize=11, labelpad=8)
    ax1.set_zlabel(r'$z_{\rm proper}$', fontsize=11, labelpad=8)
    
    # Fixed limits for smooth animation
    max_a = a_of_t(3.0, model=model)
    lim = max_a * 1.1
    ax1.set_xlim([-lim, lim])
    ax1.set_ylim([-lim, lim])
    ax1.set_zlim([-lim, lim])
    ax1.set_box_aspect([1,1,1])
    
    # Add annotation
    ax1.text2D(0.05, 0.95, "Grid stretches\nwith expansion", 
              transform=ax1.transAxes, fontsize=10,
              bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
              verticalalignment='top')
    
    # ===== RIGHT: COMOVING COORDINATES =====
    ax2 = fig.add_subplot(122, projection='3d')
    
    # Draw unit sphere surface
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
    
    ax2.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.15, color='lightgreen',
                     linewidth=0, antialiased=True, shade=False)
    
    # Draw coordinate grid (fixed)
    for x, y, z in lat_lines:
        ax2.plot(x, y, z, color='#2ECC71', alpha=0.4, linewidth=1.2)
    for x, y, z in lon_lines:
        ax2.plot(x, y, z, color='#2ECC71', alpha=0.4, linewidth=1.2)
    
    # Draw galaxy (comoving position)
    ax2.scatter([x_com], [y_com], [z_com], color='#FF6B35', s=300,
               marker='*', edgecolors='white', linewidths=2, zorder=10)
    
    # Draw line from center to galaxy
    ax2.plot([0, x_com], [0, y_com], [0, z_com],
            color='#FF6B35', linewidth=2, linestyle='--', alpha=0.6)
    
    # Labels and formatting
    ax2.set_title(r"$\mathbf{Comoving}$ $\mathbf{Coordinates}$" + f"\n" +
                 r"Radius $= 1$ (fixed)",
                 fontsize=14, fontweight='bold', pad=15)
    ax2.set_xlabel(r'$\chi_x$', fontsize=11, labelpad=8)
    ax2.set_ylabel(r'$\chi_y$', fontsize=11, labelpad=8)
    ax2.set_zlabel(r'$\chi_z$', fontsize=11, labelpad=8)
    
    # Fixed limits
    ax2.set_xlim([-1.2, 1.2])
    ax2.set_ylim([-1.2, 1.2])
    ax2.set_zlim([-1.2, 1.2])
    ax2.set_box_aspect([1,1,1])
    
    # Add annotation
    motion_text = "Fixed grid\n"
    if abs(theta_pec) < 0.01 and abs(phi_pec) < 0.01:
        motion_text += "Galaxy stationary"
    else:
        motion_text += "Galaxy drifts"
    
    ax2.text2D(0.05, 0.95, motion_text,
              transform=ax2.transAxes, fontsize=10,
              bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
              verticalalignment='top')
    
    # Set viewing angle
    ax1.view_init(elev=20, azim=45)
    ax2.view_init(elev=20, azim=45)
    
    plt.suptitle(f"Time: $t = {t:.2f}$  |  Scale Factor: $a(t) = {a:.2f}$  |  Model: {model}",
                fontsize=13, y=0.98)
    
    plt.tight_layout()
    plt.show()

# ===== WIDGETS FOR 3D SPHERE =====
play_sphere = widgets.Play(
    value=0,
    min=-100,
    max=400,
    step=1,
    interval=80,  # Slower for 3D rendering
    description="Play",
    disabled=False
)

t_sphere_slider = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=3.0,
    step=0.01,
    description="Time (t)",
    readout_format=".2f",
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

widgets.jslink((play_sphere, 'value'), (t_sphere_slider, 'value'))

model_sphere_dropdown = widgets.Dropdown(
    options=[("Toy ΛCDM-ish", "matter+Lambda"),
             ("Matter-dominated", "matter"),
             ("de Sitter (exponential)", "deSitter"),
             ("Linear expansion", "linear")],
    value="matter+Lambda",
    description="Model:",
    style={'description_width': '80px'}
)

theta_pec_slider = widgets.FloatSlider(
    value=0.0,
    min=-0.5,
    max=0.5,
    step=0.01,
    description="θ_pec (lat):",
    readout_format=".2f",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

phi_pec_slider = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description="φ_pec (lon):",
    readout_format=".2f",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Preset buttons for sphere
button_sphere_static = widgets.Button(description="Stationary Galaxy", button_style='info')
button_sphere_drift_lat = widgets.Button(description="Drift Northward", button_style='success')
button_sphere_drift_lon = widgets.Button(description="Drift Eastward", button_style='warning')
button_sphere_reset = widgets.Button(description="Reset", button_style='danger')

def on_sphere_static_click(b):
    theta_pec_slider.value = 0.0
    phi_pec_slider.value = 0.0
    t_sphere_slider.value = 0.0

def on_sphere_drift_lat_click(b):
    theta_pec_slider.value = -0.2  # Negative = toward north pole
    phi_pec_slider.value = 0.0
    t_sphere_slider.value = 0.0

def on_sphere_drift_lon_click(b):
    theta_pec_slider.value = 0.0
    phi_pec_slider.value = 0.3
    t_sphere_slider.value = 0.0

def on_sphere_reset_click(b):
    theta_pec_slider.value = 0.0
    phi_pec_slider.value = 0.0
    t_sphere_slider.value = 0.0
    model_sphere_dropdown.value = "matter+Lambda"

button_sphere_static.on_click(on_sphere_static_click)
button_sphere_drift_lat.on_click(on_sphere_drift_lat_click)
button_sphere_drift_lon.on_click(on_sphere_drift_lon_click)
button_sphere_reset.on_click(on_sphere_reset_click)

# Layout for sphere viz
ui_sphere = widgets.VBox([
    widgets.HTML("<h3>🌐 3D Expanding Sphere with Coordinate Grids</h3>"),
    widgets.HBox([play_sphere, t_sphere_slider]),
    widgets.HBox([model_sphere_dropdown, theta_pec_slider, phi_pec_slider]),
    widgets.HTML("<b>Quick Presets:</b>"),
    widgets.HBox([button_sphere_static, button_sphere_drift_lat, button_sphere_drift_lon, button_sphere_reset]),
])

out_sphere = widgets.Output()

def _update_sphere(*args):
    with out_sphere:
        clear_output(wait=True)
        plot_expanding_sphere_3d(
            t=t_sphere_slider.value,
            model=model_sphere_dropdown.value,
            theta_pec=theta_pec_slider.value,
            phi_pec=phi_pec_slider.value
        )

# Observe changes
for w in [t_sphere_slider, model_sphere_dropdown, theta_pec_slider, phi_pec_slider]:
    w.observe(_update_sphere, names="value")

display(ui_sphere, out_sphere)
_update_sphere()

Output()